In [ ]:
#importing necessary files 
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver 
import pandas as pd
from google.colab import drive
import warnings
warnings.filterwarnings("ignore")

#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=options)


Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:14 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
import pandas as pd
import re,string
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from patsy import dmatrices
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
import matplotlib.pyplot as plt
import warnings
%pylab inline
import numpy as np
from google.colab import drive
from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances
drive.mount('/drive')  

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Populating the interactive namespace from numpy and matplotlib
Mounted at /drive


In [ ]:
PwC_data = pd.read_csv('/drive/My Drive/PwC_Reviews.csv')
PwC_data.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons
0,Amazing place to grow and develop,"Sep 26, 2021",5.0,Former,Senior Management Consultant,NaN,Recommend,Business Outlook,Share,Focused a lot of early career development. Ama...,"Long hours, but honestly, not deal-breaking."
1,It's luck of the draw,"Sep 27, 2021",3.0,Current,International Tax Associate,NaN,Recommend,Business Outlook,Share,"The people, you will gain a ton of experience ...",Although the hours are flexible they are still...
2,Lots of politics,"Sep 28, 2021",4.0,Former,SAP Consultant,NaN,Recommend,Business Outlook,Share,Opportunity to work with lots of bright talent...,Very negative experience with a manager with z...
3,Its a massive ship,"Sep 30, 2021",3.0,Former,International Tax Associate,NaN,Recommend,Business Outlook,Share,"They are trying to change culture, and you can...",They are still a business and you are replacea...
4,PwC life,"Sep 21, 2021",4.0,Current,Risk and Regulatory Advisory Senior Associate,NaN,Recommend,Business Outlook,Share,Good pay and benefits Career opportunities are...,You work a lot. The firm preaches work life ba...


## Pros
###Step 1: pros/cons data pre-processing

In [ ]:
#clean punctuation and white spaces
PwC_data["pros_clean"] = PwC_data.Pros.apply(lambda x:re.sub(r'[^\w\s]', ' ', str(x).lower()))
PwC_data["pros_clean"] = PwC_data.pros_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
PwC_data["pros_clean"] = PwC_data.pros_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
PwC_data["pros_clean"] = PwC_data.pros_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

PwC_data["cons_clean"] = PwC_data.Cons.apply(lambda x:re.sub(r'[^\w\s]', ' ', str(x).lower()))
PwC_data["cons_clean"] = PwC_data.cons_clean.apply(lambda x:re.sub(r'(\\n+)', ' ', x))
PwC_data["cons_clean"] = PwC_data.cons_clean.apply(lambda x:re.sub(r'^b[\'\"\'"\"''\s]', '', x.lower()))
PwC_data["cons_clean"] = PwC_data.cons_clean.apply(lambda x:re.sub(r'\s+', ' ', x))

#tokenizing and removing stop words
stop = set(stopwords.words('english'))
punc = string.punctuation
PwC_data['pros_clean'] = PwC_data.pros_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
PwC_data['pros_clean'] = PwC_data['pros_clean'].apply(lambda x: [word for word in x if word not in punc])
PwC_data['cons_clean'] = PwC_data.cons_clean.apply(lambda x: [word for word in word_tokenize(x) if word not in stop])
PwC_data['cons_clean'] = PwC_data['cons_clean'].apply(lambda x: [word for word in x if word not in punc])

PwC_data.head()

,Title,Date Written,Rating,Current/Former,Job Title,Location,Recommendation?,Outlook,Main Text,Pros,Cons,pros_clean,cons_clean
0,Amazing place to grow and develop,"Sep 26, 2021",5.0,Former,Senior Management Consultant,NaN,Recommend,Business Outlook,Share,Focused a lot of early career development. Ama...,"Long hours, but honestly, not deal-breaking.","[focused, lot, early, career, development, ama...","[long, hours, honestly, deal, breaking]"
1,It's luck of the draw,"Sep 27, 2021",3.0,Current,International Tax Associate,NaN,Recommend,Business Outlook,Share,"The people, you will gain a ton of experience ...",Although the hours are flexible they are still...,"[people, gain, ton, experience, fast, rate, fl...","[although, hours, flexible, still, ridiculousl..."
2,Lots of politics,"Sep 28, 2021",4.0,Former,SAP Consultant,NaN,Recommend,Business Outlook,Share,Opportunity to work with lots of bright talent...,Very negative experience with a manager with z...,"[opportunity, work, lots, bright, talented, pe...","[negative, experience, manager, zero, people, ..."
3,Its a massive ship,"Sep 30, 2021",3.0,Former,International Tax Associate,NaN,Recommend,Business Outlook,Share,"They are trying to change culture, and you can...",They are still a business and you are replacea...,"[trying, change, culture, find, people, care]","[still, business, replaceable, burnt, matter, ..."
4,PwC life,"Sep 21, 2021",4.0,Current,Risk and Regulatory Advisory Senior Associate,NaN,Recommend,Business Outlook,Share,Good pay and benefits Career opportunities are...,You work a lot. The firm preaches work life ba...,"[good, pay, benefits, career, opportunities, p...","[work, lot, firm, preaches, work, life, balanc..."


### Step 2: get pros/cons attributes

In [ ]:
#get POS
get_pros = PwC_data['pros_clean'].apply(lambda x: [word for word in nltk.pos_tag(x)])
get_cons = PwC_data['cons_clean'].apply(lambda x: [word for word in nltk.pos_tag(x)])

pro_adj = []
con_adj = []

def get_adj(empty_list,pos_data):
    for review in pos_data:
        for word,pos in review:
            if pos == 'JJ' or pos == 'JJR' or pos == 'JJS': # if the POS-tag is adjective
                empty_list.append(word)
get_adj(pro_adj,get_pros)
get_adj(con_adj,get_cons)

In [ ]:
from collections import Counter
Counter(pro_adj).most_common(50)

[('great', 777),
 ('good', 489),
 ('smart', 88),
 ('flexible', 81),
 ('nice', 73),
 ('different', 73),
 ('big', 53),
 ('many', 49),
 ('interesting', 45),
 ('best', 43),
 ('professional', 41),
 ('new', 41),
 ('excellent', 36),
 ('salary', 34),
 ('high', 34),
 ('competitive', 29),
 ('top', 29),
 ('decent', 27),
 ('young', 27),
 ('strong', 26),
 ('supportive', 25),
 ('friendly', 23),
 ('learn', 23),
 ('large', 23),
 ('various', 22),
 ('digital', 19),
 ('available', 19),
 ('solid', 18),
 ('busy', 18),
 ('hard', 18),
 ('diverse', 18),
 ('overall', 17),
 ('resume', 17),
 ('easy', 17),
 ('willing', 17),
 ('corporate', 16),
 ('fantastic', 15),
 ('wide', 15),
 ('helpful', 14),
 ('awesome', 14),
 ('short', 14),
 ('right', 13),
 ('internal', 13),
 ('long', 13),
 ('inclusive', 13),
 ('technical', 12),
 ('generous', 12),
 ('able', 12),
 ('global', 12),
 ('free', 12)]

In [ ]:
#manually took some attributes from the top frequent pros words
pros_attribute = ['great','smart','flexible','nice','different','interesting','best','professional','excellent','top','young',
                  'strong','friendly','positive','different','professional','available','solid','diverse',
                  'willing','fantastic','helpful', 'awesome', 'inclusive', 'generous', 'able','global']

In [ ]:
Counter(con_adj).most_common(50)

[('long', 420),
 ('busy', 264),
 ('high', 97),
 ('low', 87),
 ('hard', 75),
 ('much', 68),
 ('great', 65),
 ('difficult', 60),
 ('salary', 59),
 ('good', 59),
 ('big', 58),
 ('bad', 56),
 ('poor', 56),
 ('many', 44),
 ('little', 43),
 ('competitive', 41),
 ('new', 33),
 ('large', 31),
 ('tough', 31),
 ('lower', 30),
 ('stressful', 29),
 ('political', 21),
 ('public', 21),
 ('due', 20),
 ('internal', 20),
 ('best', 19),
 ('underpaid', 19),
 ('top', 18),
 ('less', 17),
 ('overtime', 17),
 ('senior', 17),
 ('easy', 17),
 ('personal', 16),
 ('certain', 16),
 ('enough', 15),
 ('terrible', 14),
 ('heavy', 14),
 ('intense', 14),
 ('first', 14),
 ('work', 13),
 ('limited', 13),
 ('horrible', 13),
 ('better', 13),
 ('repetitive', 12),
 ('slow', 12),
 ('least', 12),
 ('full', 11),
 ('last', 11),
 ('overall', 11),
 ('huge', 11)]

In [ ]:
#manually took some attributes from the top frequent cons words
cons_attribute = ['long','busy','hard','difficult','bad','poor','competitve','tough','lower','stressful','political'
                 ,'underpaid','less','overtime','personal','terrible','heavy','intense', 'limited', 'horrible', 'repetitive','slow', 'last']



### Step 3: lemmentize -> get replacement

In [ ]:
replacement = {'work_life_balance':['time','life','balance','sabbatical','sabbaticals','focus','hour','day','health','flexible','week'
                                    ,'vacation','schedule','overtime'],
                'culture_value':['people','culture','team','care','value','product','coworkers','atmosphere','competitive'
                                ,'family','collaboration','respect','community','colleague','supportive','vision','diversity'],
                'career_opportunity':['opportunity','learn','industry','career','license','training','train','growth','grow'
                                    ,'level','position','development','advancement','advance','study','build','skill','resource'
                                    ,'education','potential'],
                'company_benefit':['company','benefit','pay','financial','financially','provide','salary','bonus','offer'
                                  ,'401k','package','stock','compensation','invest','investment','money','performance','reward'
                                  ,'retirement','promote','insurance'],
                'senior_management':['place','environment','management','help','manager','experience','match'
                                                ,'plan','office','support','location','leadership','treat','helpful','senior'
                                                ,'manage','leader','communication']}

In [ ]:
def getKeysByValue(dictOfElements, valueToFind):
    for k,v  in dictOfElements.items():
        if valueToFind in v:
            return(k)
    return  valueToFind

def replace_attributes(s):
    return([getKeysByValue(replacement,y) for y in s])

In [ ]:
PwC_data['pros_replace'] = PwC_data['pros_clean'].map(replace_attributes)
PwC_data['cons_replace'] = PwC_data['cons_clean'].map(replace_attributes)

In [ ]:
PwC_data['pros_replace'] [0]

['focused',
 'lot',
 'early',
 'career_opportunity',
 'career_opportunity',
 'amazing',
 'senior_management',
 'career_opportunity',
 'strategic',
 'thinking',
 'problem',
 'solving',
 'quantitative',
 'skills']

### Step 4: Lift Score

In [ ]:
def ratio(x,y):
    if x==0:
        return float(y)
    if y==0:
        return float(x)
    return(float(x)*float(y))

def get_lift(a,b,tokenized_data):
    '''Function to calculate lift scores given any two words from a list of tokenized words'''
    if (a==b):
        return 1
    p_a = len([i for i in tokenized_data if a in i])
    p_b = len([i for i in tokenized_data if (b in i)])
    p_a_b = len([i for i in tokenized_data if a in i if b in i])
    n = len(tokenized_data)
    return float(float(n)*float(p_a_b)/ratio(p_a,p_b))

In [ ]:
pros_list = replacement.keys()
lift_score = [get_lift(x,y,PwC_data.pros_replace) for x in pros_attribute for y in pros_list]
formatted_lift_score = [ round(elem,2) for elem in lift_score ]

In [ ]:
pd.DataFrame(reshape(formatted_lift_score,(len(pros_attribute),len(pros_list))),index =pros_attribute , columns = pros_list )

,work_life_balance,culture_value,career_opportunity,company_benefit,senior_management
great,0.83,1.18,1.04,1.05,1.31
smart,0.90,1.99,1.33,0.74,0.92
flexible,0.00,0.00,0.00,0.00,0.00
nice,0.86,1.58,0.80,1.31,1.26
different,0.98,0.86,1.54,0.50,1.13
interesting,0.47,1.01,0.62,0.89,0.64
best,0.86,1.14,1.46,1.53,1.04
professional,0.98,0.90,2.09,0.74,1.48
excellent,1.23,1.06,1.27,2.03,1.12
top,0.75,0.97,0.84,1.43,1.03


In [ ]:
cons_list = replacement.keys()
lift_score = [get_lift(x,y,PwC_data.cons_replace) for x in cons_attribute for y in cons_list]
formatted_lift_score = [ round(elem,2) for elem in lift_score ]

In [ ]:
pd.DataFrame(reshape(formatted_lift_score,(len(cons_attribute),len(cons_list))),index =cons_attribute , columns = cons_list )

,work_life_balance,culture_value,career_opportunity,company_benefit,senior_management
long,0.60,0.67,0.69,0.92,0.67
busy,1.04,0.59,0.66,0.60,0.51
hard,1.47,1.42,1.20,1.05,0.98
difficult,2.11,1.13,1.34,0.81,1.37
bad,1.62,2.56,0.57,1.56,1.42
poor,2.01,2.04,2.42,2.26,2.28
competitve,0.00,0.00,0.00,0.00,0.00
tough,1.42,1.61,0.32,0.93,1.30
lower,1.05,0.30,3.52,3.42,0.96
stressful,0.98,0.83,0.66,0.32,1.79


### Lift for mission Statement

In [ ]:
PwC_values = ['trust','integrity','innovation', 'improving', "caring", "together",'reimagine']

In [ ]:
mission_lift = [get_lift(x,y,PwC_data.pros_replace) for x in PwC_values for y in pros_list]
formatted_lift_score = [ round(elem,2) for elem in mission_lift ]

In [ ]:
# Don't know why there're many zeros... Maybe we should just go with more general words?
pd.DataFrame(reshape(formatted_lift_score,(len(PwC_values),len(pros_list))),index =PwC_values , columns = pros_list)

,work_life_balance,culture_value,career_opportunity,company_benefit,senior_management
trust,5.37,2.47,2.38,4.09,2.44
integrity,0.00,2.47,0.00,0.00,3.26
innovation,0.00,1.23,1.59,0.00,1.63
improving,0.00,0.00,0.00,0.00,0.00
caring,0.80,1.92,1.41,0.00,0.72
together,7.16,1.23,0.00,0.00,0.00
reimagine,0.00,0.00,0.00,0.00,0.00


-- High lifts FOR PROS

* together & work_life_balance (7.16)
* trust & work_life_balance (5.37)
* together & company_benefit (4.09)
* integrity & senior_management (3.26)

#### For Cons

In [ ]:
mission_lift = [get_lift(x,y,PwC_data.cons_replace) for x in PwC_values for y in cons_list]
formatted_lift_score = [ round(elem,2) for elem in mission_lift ]

In [ ]:
pd.DataFrame(reshape(formatted_lift_score,(len(PwC_values),len(cons_list))),index =PwC_values , columns = cons_list )

,work_life_balance,culture_value,career_opportunity,company_benefit,senior_management
trust,3.66,8.33,0.00,0.00,0.00
integrity,0.00,0.00,0.00,0.00,0.00
innovation,0.00,4.16,4.93,0.00,0.00
improving,1.83,0.00,0.00,0.00,1.68
caring,3.66,8.33,0.00,4.78,6.72
together,0.00,0.00,0.00,0.00,0.00
reimagine,0.00,0.00,0.00,0.00,0.00


-- high lifts FOR CONS

* trust & culture_value (8.33)
* innovation & career_opportunity (4.93)
* caring & culture_value (8.33)